<a href="https://colab.research.google.com/github/cristianjosemezamamani-max/estadistica_computacional/blob/main/ejercicio_practico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
from geopy.distance import geodesic

# ============================================================
# 1) Dataset de lugares turísticos - CUSCO, PERÚ
# ============================================================

lugares_cusco = {
    'id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    'lugar': [
        'Plaza de Armas', 'Qorikancha', 'San Blas', 'Sacsayhuamán', 'Qenqo',
        'Puka Pukara', 'Tambomachay', 'Cristo Blanco', 'San Cristóbal', 'Mercado San Pedro'
    ],
    'lat': [
        -13.5164, -13.5196, -13.5134, -13.5085, -13.5058,
        -13.4952, -13.4921, -13.5097, -13.5121, -13.5227
    ],
    'lng': [
        -71.9785, -71.9752, -71.9763, -71.9827, -71.9863,
        -72.0047, -72.0089, -71.9697, -71.9812, -71.9766
    ]
}

df = pd.DataFrame(lugares_cusco)
print("Lugares turísticos cargados:")
print(df)

# ============================================================
# 2) Crear matriz de distancias geográficas (km)
# ============================================================

def distancia(a, b):
    return geodesic((a["lat"], a["lng"]), (b["lat"], b["lng"])).km

n = len(df)
matriz_dist = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        if i != j:
            matriz_dist[i, j] = distancia(df.iloc[i], df.iloc[j])

print("\nMatriz de distancias creada correctamente!")

# ============================================================
# 3) Nearest Neighbor (Ciudad inicial = 0)
# ============================================================

def nearest_neighbor(dist_matrix, inicio=0):
    ruta = [inicio]
    actual = inicio
    costo = 0

    while len(ruta) < len(dist_matrix):
        no_visitadas = [j for j in range(n) if j not in ruta]
        siguiente = min(no_visitadas, key=lambda x: dist_matrix[actual][x])
        costo += dist_matrix[actual][siguiente]
        ruta.append(siguiente)
        actual = siguiente

    ruta.append(inicio)
    costo += dist_matrix[actual][inicio]
    return ruta, costo

ruta_greedy, costo_greedy = nearest_neighbor(matriz_dist)

print("\nRecorrido NN:", ruta_greedy)
print("Distancia total NN: {:.2f} km".format(costo_greedy))

# ============================================================
# 4) Mejoramiento con 2-Opt
# ============================================================

def total_dist(ruta):
    return sum(matriz_dist[ruta[i]][ruta[i+1]] for i in range(len(ruta)-1))

def two_opt(ruta):
    mejor = ruta[:]
    mejor_dist = total_dist(mejor)
    mejora = True

    while mejora:
        mejora = False
        for i in range(1, len(ruta) - 2):
            for j in range(i + 1, len(ruta) - 1):
                nueva = mejor[:i] + mejor[i:j][::-1] + mejor[j:]
                nueva_dist = total_dist(nueva)
                if nueva_dist < mejor_dist:
                    mejor = nueva
                    mejor_dist = nueva_dist
                    mejora = True
    return mejor, mejor_dist

ruta_opt, costo_opt = two_opt(ruta_greedy)

print("\nRecorrido optimizado:", ruta_opt)
print("Distancia total optimizada: {:.2f} km".format(costo_opt))

# ============================================================
# 5) Mapa interactivo Folium - Ruta Optimizada
# ============================================================

def mapa_ruta(df, ruta, titulo):
    mapa = folium.Map(location=[-13.5164, -71.9785], zoom_start=13)

    for i in df.index:
        folium.Marker(
            [df.iloc[i]["lat"], df.iloc[i]["lng"]],
            popup=df.iloc[i]["lugar"],
            icon=folium.Icon(color="red" if i == 0 else "blue")
        ).add_to(mapa)

    for i in range(len(ruta)-1):
        a = df.iloc[ruta[i]]
        b = df.iloc[ruta[i+1]]
        folium.PolyLine([(a["lat"], a["lng"]), (b["lat"], b["lng"])],
                        weight=4).add_to(mapa)

    return mapa

display(mapa_ruta(df, ruta_opt, "Ruta Optimizada - Cusco"))

# ============================================================
# 6) Comparación de Métricas
# ============================================================

print("\nRESULTADOS:")
print(f"Nearest Neighbor: {costo_greedy:.2f} km")
print(f"2-Opt Optimizado: {costo_opt:.2f} km")
print(f"Mejora: {costo_greedy - costo_opt:.2f} km")
print(f"Reducción: {((costo_greedy - costo_opt)/costo_greedy)*100:.2f}%")



Lugares turísticos cargados:
   id              lugar      lat      lng
0   0     Plaza de Armas -13.5164 -71.9785
1   1         Qorikancha -13.5196 -71.9752
2   2           San Blas -13.5134 -71.9763
3   3       Sacsayhuamán -13.5085 -71.9827
4   4              Qenqo -13.5058 -71.9863
5   5        Puka Pukara -13.4952 -72.0047
6   6        Tambomachay -13.4921 -72.0089
7   7      Cristo Blanco -13.5097 -71.9697
8   8      San Cristóbal -13.5121 -71.9812
9   9  Mercado San Pedro -13.5227 -71.9766

Matriz de distancias creada correctamente!

Recorrido NN: [0, 2, 8, 3, 4, 7, 1, 9, 5, 6, 0]
Distancia total NN: 14.47 km

Recorrido optimizado: [0, 9, 1, 7, 2, 8, 3, 4, 5, 6, 0]
Distancia total optimizada: 11.77 km



RESULTADOS:
Nearest Neighbor: 14.47 km
2-Opt Optimizado: 11.77 km
Mejora: 2.70 km
Reducción: 18.64%
